In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests, openpyxl
import io
from PIL import Image
import time

# Path to the chrome webdriver on my PC
PATH = "C:\\Users\\niloo\\OneDrive\\Desktop\\ChromeDriver\\chromedriver.exe"

# Create a new Excel workbook and sheet
excel = openpyxl.Workbook()
sheet = excel.active
sheet.title = 'Adidas Scraped Data'
sheet.append(['Title','Caterory','Price','Image Path','Image URL'])

wd = webdriver.Chrome(PATH)

# Functions to scroll the page up and down smoothly to prevent the price boxes disappear once the item is not shown in
# the part we focus
def scroll_down(driver):
    bottom=False
    a=0
    while not bottom:
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {a});")
        if a > 2.5/4*new_height:
            bottom=True
        a+=5
def scroll_up(driver):
    bottom=False
    a=driver.execute_script("return document.body.scrollHeight")
    while not bottom:
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {a});")
        if a < 10:
            bottom=True
        a-=5


def get_data_from_adidas(wd, delay, max_images, n, sheet):

    url = "https://www.adidas.ca/en/men-shirts"
    if n>0:
        url = url +"?start="+str(n*48)
        print('*********************************\n Page '+str(n+1)+' \n***********************************')
    wd.get(url)
    scroll_up(wd)
    scroll_down(wd)
    image_urls = []

    while len(image_urls) < max_images:

        thumbnails = wd.find_elements(By.CSS_SELECTOR, ".product-card-image.glass-product-card__image")
        if len(thumbnails)<max_images:
            max_images = len(thumbnails)
        prices = wd.find_elements(By.CLASS_NAME, "badge-container___1TJjk")
        titles = wd.find_elements(By.CLASS_NAME, "glass-product-card__title")
        categories = wd.find_elements(By.CLASS_NAME, "glass-product-card__category")
        l = len(image_urls)
        for i,(price,image,title,categ) in enumerate(zip(prices[len(image_urls) :max_images],thumbnails[len(image_urls) :max_images],titles[len(image_urls) :max_images],categories[len(image_urls) :max_images])):
            if image.get_attribute('src') in image_urls:
                max_images += 1
                break

            if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                image_urls.append(image.get_attribute('src'))
                print('\n'+f"Found {len(image_urls)}")
                print(price.text)
                print(title.text)
                print(categ.text)
                sheet.append([title.text, categ.text, price.text, 'images Adidas/'+str(l+i+n*48)+".jpg", image.get_attribute('src')])

    return image_urls


def download_image(download_path, url, file_name):
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = download_path + file_name

        with open(file_path, "wb") as f:
            image.save(f, "JPEG")

    except Exception as e:
        print('FAILED -', e)
        
# Finding the number of pages
url = "https://www.adidas.ca/en/men-shirts"
wd.get(url)

element = wd.execute_script("return document.querySelector('#main-content > div > div:nth-child(3) > div > div > div > div:nth-child(2) > div > div > div > div:nth-child(2) > span').textContent.trim()")
number = element.strip()
number_of_pages = int(number.split('/')[-1])

for n in range(number_of_pages):
    delay = 1
    urls = get_data_from_adidas(wd, delay, 48, n, sheet) # 48 is the number of items in each page

    for i, url in enumerate(urls):
        download_image("C:/Users/niloo/OneDrive/Desktop/ChromeDriver/images Adidas/", url, str(i+n*48) + ".jpg")

    
excel.save('Adidas Scraped Data.xlsx')


Found 1
C$ 42
Adicolor Classics 3-Stripes Tee
Men Originals

Found 2
C$ 50
Adicolor Classics 3-Stripes Long Sleeve Tee
Men Originals

Found 3
C$ 42
Adicolor Classics 3-Stripes Tee
Men Originals

Found 4
C$ 42
Essentials Single Jersey 3-Stripes Tee
Men Sportswear

Found 5
C$ 42
Adicolor Classics 3-Stripes Tee
Men Originals

Found 6
C$ 60
Performance Primegreen Polo Shirt
Men Golf

Found 7
C$ 70
Designed 4 Training HEAT.RDY HIIT Training Tee
Men Training

Found 8
C$ 65
3-Stripes Polo Shirt
Men Golf

Found 9
C$ 65
3-Stripes Polo Shirt
Men Golf

Found 10
C$ 45
All SZN Graphic Tee
Men Sportswear

Found 11
C$ 40
ALL SZN Tee
Men Sportswear

Found 12
C$ 65
3-Stripes Polo Shirt
Men Golf

Found 13
C$ 40
Messi Graphic Tee
Men Soccer

Found 14
C$ 60
Performance Primegreen Polo Shirt
Men Golf

Found 15
C$ 40
Trefoil Essentials Tee
Men Originals

Found 16
C$ 40
Trefoil Essentials Tee
Men Originals

Found 17
C$ 42
Adicolor Classics 3-Stripes Tee
Men Originals

Found 18
C$ 50
Adicolor Classics 3-Stri


Found 46
C$ 31.25
Train Essentials Feelready Training Tee
Men Gym & Training

Found 47
C$ 40
Henry Jones Paris Tee
Men Originals

Found 48
C$ 110
Blokepop Rugby Polo Shirt
Men Originals
*********************************
 Page 4 
***********************************

Found 1
C$ 52
Designed for Training Tee
Men Training

Found 2
C$ 40
Messi Player Tee
Men Soccer

Found 3
C$ 60
AAC Tee
Men Originals

Found 4
C$ 40
Shmoofoil Tear Short Sleeve Tee
Men Originals

Found 5
C$ 38
Liquid Foil Badge of Sport Graphic Tee
Men Sportswear

Found 6
C$ 50
AEROREADY Yoga Tee
Men Training

Found 7
C$ 31.25
Train Essentials Feelready Training Sleeveless Tee
Men Gym & Training

Found 8
C$ 50
Adicross Drop Two Tee (Gender Neutral)
Unisex Golf

Found 9
C$ 60
AAC Tee
Men Originals

Found 10
C$ 75
Ottoman Stripe Polo Shirt
Men Golf

Found 11
C$ 55
Graphic Planet Tee
Men Originals

Found 12
C$ 45
adidas RIFTA Metro Short Sleeve Tee
Men Originals

Found 13
C$ 100
Go-To Camp Golf Shirt
Men Golf

Found 14
C$ 40
He

C$ 65
Yoga Training Tee
Men Studio

Found 39
C$ 60
Studio Lounge Tee
Men Sportswear

Found 40
C$ 38
Essentials Single Jersey Big Logo Tee
Men Sportswear

Found 41
C$ 100
Five Ten TrailX Long Sleeve Tee
Men Five Ten

Found 42
C$ 70
Confident Engineered Tee
Men Running

Found 43
C$ 100
Ultimate365 Tour HEAT.RDY Polo Shirt
Men Golf

Found 44
C$ 45
Sportswear Elevated Block Tee
Men Sportswear

Found 45
C$ 45
Sportswear Elevated Block Tee
Men Sportswear

Found 46
C$ 55
adidas Originals x André Saraiva Tee
Men Originals

Found 47
C$ 55
Future Icons Badge of Sport Tee
Men Sportswear

Found 48
C$ 80
Terrex Trail Running Long Sleeve Tee
Men TERREX
*********************************
 Page 7 
***********************************

Found 1
C$ 80
Terrex Trail Running Long Sleeve Tee
Men TERREX

Found 2
C$ 38
Flames Playmaker Tee
Men Hockey

Found 3
C$ 65
Lounge Tee
Men Sportswear

Found 4
C$ 55
adidas Kidcore Tee
Unisex Sportswear

Found 5
C$ 60
Designed 4 Gameday Tee
Men Sportswear

Found 6
C$ 110
Be

Men Golf

Found 34
C$ 90
Go-To Striped Golf Polo Shirt
Men Golf

Found 35
C$ 38
Trefoil Essentials Tee
Men Originals

Found 36
C$ 90
Go-To Striped Golf Polo Shirt
Men Golf

Found 37
C$ 75
Striated Polo Shirt
Men Golf

Found 38
C$ 40
Daewon's Business Short Sleeve Tee
Men Originals

Found 39
C$ 45
adidas Basketball Sleeveless Tee
Unisex Basketball

Found 40
C$ 42
Adicolor Classics 3-Stripes Tee
Men Originals

Found 41
C$ 65
Adicolor Classics Shirt
Men Originals

Found 42
C$ 85
Flower Mesh Golf Polo Shirt
Men Golf

Found 43
C$ 58
HIIT Training Tee
Men Training

Found 44
C$ 40
Bruins Blend Tee
Men Hockey

Found 45
C$ 100
Bogey Boys Polo Shirt
Men Golf

Found 46
C$ 42
Essentials Single Jersey 3-Stripes Tee
Men Sportswear

Found 47
C$ 38
Adicolor Essentials Trefoil Tee
Men Originals

Found 48
C$ 45
Golf Tee
Men Golf
*********************************
 Page 10 
***********************************

Found 1
C$ 40
ALL SZN Tee
Men Sportswear

Found 2
C$ 45
Graphics New Age Tee
Men Originals

Foun


Found 25
Sold out
Canadiens Playmaker Long Sleeve Tee
Men Sportswear

Found 26
C$ 38
Adicolor Classics Trefoil Tee
Men Originals

Found 27
C$ 40
Train Essentials Feelready Logo Training Tee
Men Gym & Training

Found 28
C$ 60
Colorblock Tee
Men Sportswear

Found 29
C$ 60
Colorblock Tee
Men Sportswear

Found 30
C$ 52
Designed for Training Tee
Men Training

Found 31
C$ 38
Adicolor Classics Trefoil Tee
Men Originals

Found 32
C$ 45
AEROREADY Designed 2 Move Feelready Sport Long Sleeve Tee
Men Training

Found 33
C$ 65
C$ 55
Workout PU Print Tee
Men Training

Found 34
C$ 60
Colorblock Tee
Men Sportswear

Found 35
C$ 50
C$ 35
D.O.N. Excellence Tee
Men Basketball

Found 36
C$ 50
Lift Your Mind Designed for Training Tank Top
Men Gym & Training

Found 37
C$ 45
Own the Run Tee
Men Running

Found 38
C$ 70
adidas x Marimekko Run Icons 3-Stripes Tee
Men Running

Found 39
C$ 65
C$ 55
Workout PU Print Tee
Men Training

Found 40
C$ 52
Designed for Training Tee
Men Training

Found 41
C$ 65
C$ 59
Design

Designed 4 Running Tee
Men Running

Found 7
C$ 110
C$ 94
Best of Adi Training Tee
Men Gym & Training

Found 8
C$ 120
C$ 102
Best of Adi Training Long Sleeve Tee
Men Gym & Training

Found 9
C$ 45
C$ 36
RIFTA City Boy Tee
Men Originals

Found 10
C$ 50
C$ 40
Pogba Icon Graphic Tee
Men Soccer

Found 11
C$ 40
C$ 34
adidas Power Logo Graphic Tee
Men Sportswear

Found 12
C$ 90
C$ 72
adidas Rekive Woven Long Sleeve Top
Men Originals

Found 13
C$ 45
C$ 36
Streetball Allover Print Graphic Tee
Men Originals

Found 14
C$ 65
C$ 59
HEAT.RDY HIIT Elevated Training Tank Top
Men Training

Found 15
C$ 55
C$ 44
Adicolor Seasonal Archive Tee
Men Originals

Found 16
C$ 52
C$ 47
Future Icons Badge of Sport Tee
Men Sportswear

Found 17
C$ 45
C$ 38
Future Icons Graphic Short Sleeve Tee
Men Sportswear

Found 18
C$ 45
C$ 36
RIFTA City Boy Tee
Men Originals

Found 19
C$ 48
C$ 41
Workout Base Tee
Men Training

Found 20
C$ 65
C$ 59
HEAT.RDY HIIT Elevated Training Tank Top
Men Training

Found 21
C$ 80
C$ 64
Adicolo